In [36]:
# The code was removed by Watson Studio for sharing.

<h1>Coursera Capstone Project</h1>
<hr>
<h2>Exploring location data use</h2>

Import the required libraries

In [1]:
#!conda install --override-channels -c conda-forge geopy --yes 
!conda install --override-channels -c conda-forge/noarch geopy
!conda install --override-channels -c conda-forge/noarch folium
#!conda install -c conda-forge folium 
print('Folium installed')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geopy-1.22.0               |     pyh9f0ad1d_0          63 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          97 KB

The following NEW packages will be INSTALLED:

    geographiclib: 1.50-py_0           conda-forge
    geopy:         1.22.0-pyh9f0ad1d_0 conda-forge


geopy-1.22.0         | 63 KB     | ##################################### | 100% 
geographiclib-1.50   | 34 KB     | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: done
Solving environment: done

## Packa

In [2]:
!pip install lxml html5lib beautifulsoup4

In [3]:
import pandas as pd
import numpy as np
import requests # library to handle requests
import random # library for random number generation
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

import folium # plotting library

print('Libraries imported.')

Libraries imported.


<h2>Scrap website and retrieve table with postal codes</h2>

In [ ]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
dfs = pd.read_html(url)
df = dfs[0][['Postal Code', 'Borough', 'Neighborhood']]

<h3>Clean up dataset</h3>

In [31]:
dftoronto = df[df['Borough']!= 'Not assigned']

#Use group by with join to remove duplicate postal codes
dftoronto = dftoronto.groupby('Postal Code').agg({'Borough': ', '.join, 
                             'Neighborhood': ', '.join}).reset_index()

#Take care of Not assigned Neighborhood
dftoronto = dftoronto[dftoronto['Neighborhood']!= 'Not assigned']
dftoronto.head()

,Postal Code,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [35]:
print('Shapre of dataframe: ', dftoronto.shape)

Shapre of dataframe:  (103, 3)


In [40]:
# The code was removed by Watson Studio for sharing.

{'file_name': 'dftoronto.csv',
 'message': 'File saved to project storage.',
 'bucket_name': 'datascience-donotdelete-pr-lmuvdeywto5fgx',
 'asset_id': 'a8581b56-e2a8-4d8d-b48b-4d0d053327ad'}